In [6]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import networkx as nx
import pickle
import datetime
from datetime import timedelta

In [7]:
#A function to read in a pickle data object 
def Pickle_read(pathtopickle):
    #Read in the pickle
    filename=pathtopickle
    pickle_in=open(filename,"rb")
    Train=pickle.load(pickle_in)
    return Train

In [35]:
Timetabledata=Pickle_read("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Alt2019-09-01Timetable.pickle")
filename="/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/STANberth.pickle"
pickle_in=open(filename,"rb")
STANberth=pickle.load(pickle_in)
STANberth1=STANberth[STANberth['TD_ID']=='UR'].reset_index(drop=True)
a=STANberth1['TIPLOC'].unique()
for i in range(0,len(Timetabledata)):
    if np.isin(Timetabledata['Location'][i][0:7],a)==True:
        Timetabledata.at[i,'Path']='UR'
for i in range(0,len(Timetabledata)):
    Timetabledata.at[i,'Location']=Timetabledata['Location'][i].strip()
    for j in range(0,5):
        Timetabledata.at[i,'Location']=Timetabledata['Location'][i].rstrip(str(j))
    #Add berths and relevant offsets to the Timetabledata dataframe
Timetabledata['Berth']=np.array
Timetabledata['Offset']=np.array
my_list=[]
my_other_list=[]
for i in range(0,len(Timetabledata['Location'])):
    prep=STANberth[STANberth['TIPLOC']==Timetabledata['Location'][i]].reset_index()
    if len(prep['FROMBERTH'])>0:
        for j in range(0,len(prep['FROMBERTH'])):
            tupl=list([prep['FROMBERTH'][j],prep['TOBERTH'][j]])
            my_list=my_list+[tupl]
            my_other_list=my_other_list+[prep['STOPPINGOFFSET'][j]]
        Timetabledata.at[i,'Berth']=my_list
        Timetabledata.at[i,'Offset']=my_other_list
        my_list=[]
        my_other_list=[]
    else:
        Timetabledata.at[i,'Berth']='NaN'
        Timetabledata.at[i,'Offset']='NaN'
Timetable=Timetabledata
for i in range(0,len(Timetable)):
    if Timetable['SchedArr'][i]==' ':
        Timetable.at[i,'SchedArr']=Timetable['SchedDept'][i]
    if Timetable['SchedArr'][i]=='     ':
        Timetable.at[i,'SchedArr']=Timetable['SchedPass'][i]
    if Timetable['SchedDept'][i]==' ':
        Timetable.at[i,'SchedDept']=Timetable['SchedArr'][i]
    if Timetable['SchedDept'][i]=='     ':
        Timetable.at[i,'SchedDept']=Timetable['SchedPass'][i]
#Converts the SchedArr and SchedDept into more useful datetime objects
    
Timetable['ModTime']=Timetable['SchedArr'].str.slice(0,4)
Timetable['Half']=Timetable['SchedArr'].str.slice(4,5)
Timetable['Time']=pd.to_datetime('20190901'+Timetable['ModTime']+'00',format='%Y%m%d%H%M%S')
Timetable['ArrTime']=Timetable['Time']
for i in range(0,len(Timetable['Half'])):
    if Timetable['Half'][i]=='H':
        Timetable.at[i,'ArrTime']=Timetable['Time'][i]+timedelta(seconds=30)
Timetable['ModTime']=Timetable['SchedDept'].str.slice(0,4)
Timetable['Half']=Timetable['SchedDept'].str.slice(4,5)
Timetable['Time']=pd.to_datetime('20190901'+Timetable['ModTime']+'00',format='%Y%m%d%H%M%S')
Timetable['DeptTime']=Timetable['Time']
for i in range(0,len(Timetable['Half'])):
    if Timetable['Half'][i]=='H':
        Timetable.at[i,'DeptTime']=Timetable['Time'][i]+timedelta(seconds=30)
AltTimetable=pd.DataFrame()
grouped=Timetable.groupby('TrainUID')
for code,group in grouped:
    g=group.copy().reset_index(drop=True)
    g.at[0,'Msg_type']='LO'
    g.at[len(g)-1,'Msg_type']='LT'
    for i in range(0,len(g)-1):
        if g['Location'][i]=='PITSEA' and g['Location'][i+1]=='OCKENDN':
            g.at[i,'Msg_type']='LT'
    if len(g)>=1:
        AltTimetable=AltTimetable.append(g).reset_index(drop=True)
AltTimetable=AltTimetable[AltTimetable['Location']!='OCKENDN'].reset_index(drop=True)
STPC=Pickle_read("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/2019-09-01STPC.pickle")
STPCblank=STPC[STPC['Dateto']=='      ']
STPCblank['Dateto']=STPCblank['Datefrom']
STPC=STPC[STPC['Dateto']!='      ']
STPC=STPC.append(STPCblank).reset_index(drop=True)
STPC['Datefrom']=pd.to_datetime('20'+STPC['Datefrom'].str.slice(0,6),format='%Y%m%d')
STPC['Dateto']=pd.to_datetime('20'+STPC['Dateto'].str.slice(0,6),format='%Y%m%d')
PaltTimetable=AltTimetable[AltTimetable['STP']=='P'].reset_index(drop=True)
pickle_out=open("Provis2019-09-01Timetable.pickle","wb")
pickle.dump(PaltTimetable, pickle_out)
pickle_out=open("Provis2019-09-02Timetable.pickle","wb")
pickle.dump(PaltTimetable, pickle_out)
UniqueSTPC=STPC['TrainUID'].unique()
for i in range(0,len(UniqueSTPC)):
    if np.isin(UniqueSTPC[i],AltTimetable['TrainUID'].unique())==False:
        STPC=STPC[STPC['TrainUID']!=UniqueSTPC[i]].reset_index(drop=True)
a=STPC['Dateto'][0]-timedelta(days=7)
Dates=np.array([a])
for i in range(0,1):
    a=a+timedelta(days=1)
    Dates=np.append(Dates,a)
for i in range(0,len(STPC)):
    for j in range(0,len(Dates)):
        if STPC['Datefrom'][i]<=Dates[j]<=STPC['Dateto'][i]:
            CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle")
            CurrPro=CurrPro[CurrPro['TrainUID']!=STPC['TrainUID'][i]].reset_index(drop=True)
            pickle_out=open("Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle","wb")
            pickle.dump(CurrPro, pickle_out)
NaltTimetable=AltTimetable[AltTimetable['STP']=='N'].reset_index(drop=True)
for i in range(0,len(NaltTimetable)):
    for j in range(0,len(Dates)):
        if NaltTimetable['Datefrom'][i]<=Dates[j]<=NaltTimetable['Dateto'][i]:
            CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle")
            CurrPro=CurrPro.append(NaltTimetable.iloc[i]).reset_index(drop=True)
            pickle_out=open("Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle","wb")
            pickle.dump(CurrPro, pickle_out)
OaltTimetable=AltTimetable[AltTimetable['STP']=='O'].reset_index(drop=True)
for i in range(0,len(OaltTimetable)):
    for j in range(0,len(Dates)):
        if OaltTimetable['Datefrom'][i]<=Dates[j]<=OaltTimetable['Dateto'][i]:
            CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle")
            CurrPro=CurrPro.append(OaltTimetable.iloc[i]).reset_index(drop=True)
            pickle_out=open("Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle","wb")
            pickle.dump(CurrPro, pickle_out)
CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-01Timetable.pickle")
CurrPro['Trainruns']=CurrPro['Daysrun'].str.slice(6,7)
CurrPro=CurrPro[CurrPro['Trainruns']=='1'].reset_index(drop=True)
pickle_out=open("Provis2019-09-01Timetable.pickle","wb")
pickle.dump(CurrPro, pickle_out)

Timetabledata=Pickle_read("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/Alt2019-09-02Timetable.pickle")
for i in range(0,len(Timetabledata)):
    if np.isin(Timetabledata['Location'][i][0:7],a)==True:
        Timetabledata.at[i,'Path']='UR'
for i in range(0,len(Timetabledata)):
    Timetabledata.at[i,'Location']=Timetabledata['Location'][i].strip()
    for j in range(0,5):
        Timetabledata.at[i,'Location']=Timetabledata['Location'][i].rstrip(str(j))
    #Add berths and relevant offsets to the Timetabledata dataframe
Timetabledata['Berth']=np.array
Timetabledata['Offset']=np.array
my_list=[]
my_other_list=[]
for i in range(0,len(Timetabledata['Location'])):
    prep=STANberth[STANberth['TIPLOC']==Timetabledata['Location'][i]].reset_index()
    if len(prep['FROMBERTH'])>0:
        for j in range(0,len(prep['FROMBERTH'])):
            tupl=list([prep['FROMBERTH'][j],prep['TOBERTH'][j]])
            my_list=my_list+[tupl]
            my_other_list=my_other_list+[prep['STOPPINGOFFSET'][j]]
        Timetabledata.at[i,'Berth']=my_list
        Timetabledata.at[i,'Offset']=my_other_list
        my_list=[]
        my_other_list=[]
    else:
        Timetabledata.at[i,'Berth']='NaN'
        Timetabledata.at[i,'Offset']='NaN'
Timetable=Timetabledata
for i in range(0,len(Timetable)):
    if Timetable['SchedArr'][i]==' ':
        Timetable.at[i,'SchedArr']=Timetable['SchedDept'][i]
    if Timetable['SchedArr'][i]=='     ':
        Timetable.at[i,'SchedArr']=Timetable['SchedPass'][i]
    if Timetable['SchedDept'][i]==' ':
        Timetable.at[i,'SchedDept']=Timetable['SchedArr'][i]
    if Timetable['SchedDept'][i]=='     ':
        Timetable.at[i,'SchedDept']=Timetable['SchedPass'][i]
#Converts the SchedArr and SchedDept into more useful datetime objects
    
Timetable['ModTime']=Timetable['SchedArr'].str.slice(0,4)
Timetable['Half']=Timetable['SchedArr'].str.slice(4,5)
Timetable['Time']=pd.to_datetime('20190902'+Timetable['ModTime']+'00',format='%Y%m%d%H%M%S')
Timetable['ArrTime']=Timetable['Time']
for i in range(0,len(Timetable['Half'])):
    if Timetable['Half'][i]=='H':
        Timetable.at[i,'ArrTime']=Timetable['Time'][i]+timedelta(seconds=30)
Timetable['ModTime']=Timetable['SchedDept'].str.slice(0,4)
Timetable['Half']=Timetable['SchedDept'].str.slice(4,5)
Timetable['Time']=pd.to_datetime('20190902'+Timetable['ModTime']+'00',format='%Y%m%d%H%M%S')
Timetable['DeptTime']=Timetable['Time']
for i in range(0,len(Timetable['Half'])):
    if Timetable['Half'][i]=='H':
        Timetable.at[i,'DeptTime']=Timetable['Time'][i]+timedelta(seconds=30)
AltTimetable=pd.DataFrame()
grouped=Timetable.groupby('TrainUID')
for code,group in grouped:
    g=group.copy().reset_index(drop=True)
    g.at[0,'Msg_type']='LO'
    g.at[len(g)-1,'Msg_type']='LT'
    for i in range(0,len(g)-1):
        if g['Location'][i]=='PITSEA' and g['Location'][i+1]=='OCKENDN':
            g.at[i,'Msg_type']='LT'
    if len(g)>=1:
        AltTimetable=AltTimetable.append(g).reset_index(drop=True)
AltTimetable=AltTimetable[AltTimetable['Location']!='OCKENDN'].reset_index(drop=True)
STPC=Pickle_read("/home/mathsys1/Documents/MathSys/Network_rail_data_analysis/2019-09-02STPC.pickle")
STPCblank=STPC[STPC['Dateto']=='      ']
STPCblank['Dateto']=STPCblank['Datefrom']
STPC=STPC[STPC['Dateto']!='      ']
STPC=STPC.append(STPCblank).reset_index(drop=True)
STPC['Datefrom']=pd.to_datetime('20'+STPC['Datefrom'].str.slice(0,6),format='%Y%m%d')
STPC['Dateto']=pd.to_datetime('20'+STPC['Dateto'].str.slice(0,6),format='%Y%m%d')
PaltTimetable=AltTimetable[AltTimetable['STP']=='P'].reset_index(drop=True)
UniqueSTPC=STPC['TrainUID'].unique()
for i in range(0,len(UniqueSTPC)):
    if np.isin(UniqueSTPC[i],AltTimetable['TrainUID'].unique())==False:
        STPC=STPC[STPC['TrainUID']!=UniqueSTPC[i]].reset_index(drop=True)
for i in range(0,len(STPC)):
    for j in range(1,len(Dates)):
        if STPC['Datefrom'][i]<=Dates[j]<=STPC['Dateto'][i]:
            CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle")
            CurrPro=CurrPro[CurrPro['TrainUID']!=STPC['TrainUID'][i]].reset_index(drop=True)
            pickle_out=open("Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle","wb")
            pickle.dump(CurrPro, pickle_out)
for i in range(0,len(PaltTimetable)):
    for j in range(1,len(Dates)):
        if PaltTimetable['Datefrom'][i]<=Dates[j]<=PaltTimetable['Dateto'][i]:
            CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle")
            CurrPro=CurrPro.append(PaltTimetable.iloc[i]).reset_index(drop=True)
            pickle_out=open("Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle","wb")
            pickle.dump(CurrPro, pickle_out)
NaltTimetable=AltTimetable[AltTimetable['STP']=='N'].reset_index(drop=True)
for i in range(0,len(NaltTimetable)):
    for j in range(1,len(Dates)):
        if NaltTimetable['Datefrom'][i]<=Dates[j]<=NaltTimetable['Dateto'][i]:
            CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle")
            CurrPro=CurrPro.append(NaltTimetable.iloc[i]).reset_index(drop=True)
            pickle_out=open("Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle","wb")
            pickle.dump(CurrPro, pickle_out)
OaltTimetable=AltTimetable[AltTimetable['STP']=='O'].reset_index(drop=True)
for i in range(0,len(OaltTimetable)):
    for j in range(1,len(Dates)):
        if OaltTimetable['Datefrom'][i]<=Dates[j]<=OaltTimetable['Dateto'][i]:
            CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle")
            CurrPro=CurrPro.append(OaltTimetable.iloc[i]).reset_index(drop=True)
            pickle_out=open("Provis2019-09-"+str(Dates[j])[8:10]+"Timetable.pickle","wb")
            pickle.dump(CurrPro, pickle_out)
CurrPro=Pickle_read("/home/mathsys1/Documents/Provis2019-09-02Timetable.pickle")
CurrPro['Trainruns']=CurrPro['Daysrun'].str.slice(0,1)
CurrPro=CurrPro[CurrPro['Trainruns']=='1'].reset_index(drop=True)
pickle_out=open("Provis2019-09-02Timetable.pickle","wb")
pickle.dump(CurrPro, pickle_out)
CurrPro

/home/mathsys1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Msg_type,Location,SchedArr,SchedDept,SchedPass,PubArr,PubDept,Platform,Line,Path,...,Datefrom,Dateto,Berth,Offset,ModTime,Half,Time,ArrTime,DeptTime,Trainruns
0,LT,BARKING,1139,1139,1139,0000,0000,8,,UR,...,2019-05-20,2019-12-13,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",1139,,2019-10-01 11:39:00,2019-10-01 11:39:00,2019-10-01 11:39:00,1
1,LO,BARKING,1320H,1320H,1320H,0000,0000,8,,UR,...,2019-05-20,2019-09-20,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",1320,H,2019-10-01 13:20:00,2019-10-01 13:20:30,2019-10-01 13:20:30,1
2,LT,BARKING,1320H,1320H,1320H,0000,0000,8,,UR,...,2019-09-23,2019-12-13,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",1320,H,2019-10-01 13:20:00,2019-10-01 13:20:30,2019-10-01 13:20:30,1
3,LO,BARKING,2321,2321,2321,0000,0000,8,,UR,...,2019-05-20,2019-12-12,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",2321,,2019-10-01 23:21:00,2019-10-01 23:21:00,2019-10-01 23:21:00,1
4,LT,BARKING,2159H,2159H,2159H,0000,0000,8,,UR,...,2019-05-20,2019-12-13,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",2159,H,2019-10-01 21:59:00,2019-10-01 21:59:30,2019-10-01 21:59:30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4775,LO,BARKING,0007,0021H,,0000,0000,1,,UR,...,2019-06-03,2019-12-12,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",0021,H,2019-10-01 00:21:00,2019-10-01 00:07:00,2019-10-01 00:21:30,1
4776,LI,BARKING,0202H,0213,,0000,0000,7,,UR,...,2019-06-03,2019-12-12,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",0213,,2019-10-01 02:13:00,2019-10-01 02:02:30,2019-10-01 02:13:00,1
4777,LI,BARKING,0349H,0349H,0349H,0000,0000,7,DCL,UR,...,2019-06-03,2019-12-12,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",0349,H,2019-10-01 03:49:00,2019-10-01 03:49:30,2019-10-01 03:49:30,1
4778,LI,BARKING,0400,0400,0400,0000,0000,8,,UR,...,2019-06-03,2019-12-12,"[[0161, 0167], [0162, 0160], [0167, 0169], [01...","[0, -60, 0, 0, 0, 60, 0, 0, 60, 20, 124, 0, 0,...",0400,,2019-10-01 04:00:00,2019-10-01 04:00:00,2019-10-01 04:00:00,1
